# Run this if you want to do a clean make (Deletes the entire project file)

In [ ]:
import os
os.system('rm my-hls-test -r -d')

# Run this if you want to download the folder, which requires zipping it up

In [ ]:
!tar chvfz notebook.tar.gz *

# Run the next 2 boxes in order to create the directory and load the hls4ml library

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)
import os
os.environ['PATH'] = '/opt/Xilinx/Vivado/2019.2/bin:' + os.environ['PATH']
import hls4ml

In [ ]:
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

## If you want to use an example model, edit the fetch and open lines below
       The for-loop in this box is an example of how you could modify invidiual layers

In [ ]:
hls_config = hls4ml.utils.fetch_example_model('KERAS_conv2d_model.json')
json_file = open('KERAS_conv2d_model.json')
loaded = json_file.read()
json_file.close()
model = model_from_json(loaded)

hls_config = hls4ml.utils.config_from_keras_model(model, granularity='name')
for Layer in hls_config['LayerName'].keys():
    hls_config['LayerName'][Layer]['Strategy'] = 'Latency'
    hls_config['LayerName'][Layer]['ReuseFactor'] = 1
#If you want best numerical performance for high-accuray models, while the default latency strategy is faster but numerically more unstable
hls_config['LayerName']['dense_1_softmax']['Strategy'] = 'Stable'


In [ ]:
print(hls_config)

In [ ]:
# Create the config for the model
cfg = hls4ml.converters.create_config(backend='Vivado')
cfg['IOType']     = 'io_stream' # Must set this if using CNNs!
cfg['HLSConfig']  = hls_config
cfg['KerasModel'] = model
cfg['OutputDir']  = 'cnn/'
cfg['XilinxPart'] = 'xcu250-figd2104-2L-e'

print(cfg) #You can print it to see some default parameters

#Convert it to a hls project

hls_model = hls4ml.converters.keras_to_hls(cfg)
hls4ml.utils.plot_model(hls_model, show_shapes=True, show_precision=True, to_file=None)

## This is where the model will actually be built
    Run this once to create the my-hls-test directory, which will contain all of the C++ files, as well as weights, biases and verilog files
    If you want to change the weights and biases, run this to make sure the directory is created, then
          Go to my-hls-test/firmware/weights and replace the weights and biases with your own. 
                 Make sure you keep the rest of those files the same

In [ ]:
resultOfBuild = hls_model.build(csim=False)

# In some cases the model does not update the part, this code fixes it and re-runs the build
if (not resultOfBuild):
    with open('my-hls-test/build_prj.tcl', 'r') as file :
      filedata = file.read()

    # Replace the target string
    filedata = filedata.replace('xcku115-flvb2104-2-i', 'xcu250-figd2104-2L-e')

    # Write the file out again
    with open('my-hls-test/build_prj.tcl', 'w') as file:
      file.write(filedata)
    hls_model.build(csim=False)
